In [ ]:
import gurobipy as gp
import math
import pandas as pd

In [ ]:
IP = 0
MIP = 1
LP = 2


# model improved version of the "Binary Representation of One Factor" method
def model_one_factor_binary(N, model_type, m):
    # find smallest integer k such that N <= 2^k
    k = 0
    coeff = 1
    coeffs = []
    while coeff < N:
        coeffs.append(coeff)
        coeff *= 2
        k += 1
    
    improved_bound = math.ceil(k/2) + 1
    coeffs = coeffs[0:improved_bound]
        
        
    # Add variables
    if model_type == IP:
        X = m.addVar(vtype=gp.GRB.INTEGER, name="X")
        Y = m.addVars(1, improved_bound, vtype=gp.GRB.BINARY, name="Y")
        Z = m.addVars(1, improved_bound, vtype=gp.GRB.INTEGER, name="Z")
    elif model_type == MIP:
        X = m.addVar(vtype=gp.GRB.INTEGER, name="X")
        Y = m.addVars(1, improved_bound, vtype=gp.GRB.BINARY, name="Y")
        Z = m.addVars(1, improved_bound, vtype=gp.GRB.CONTINUOUS, name="Z")
    elif model_type == LP:
        X = m.addVar(vtype=gp.GRB.CONTINUOUS, name="X")
        Y = m.addVars(1, improved_bound, vtype=gp.GRB.CONTINUOUS, name="Y")
        Z = m.addVars(1, improved_bound, vtype=gp.GRB.CONTINUOUS, name="Z")

        
    # Set objective
    m.setObjective(0, gp.GRB.MAXIMIZE)

    
    # Add constraints
    Z_list = []
    for i in range(improved_bound):
        Z_list.append(Z[0,i])

    m.addConstr(N == gp.LinExpr(coeffs, Z_list))

    m.addConstr(2 <= X)
    m.addConstr(X <= math.floor(N/2))

    for i in range(improved_bound):
        m.addConstr(Z[0,i] <= N*Y[0,i])
        m.addConstr(Z[0,i] <= X)
        m.addConstr(X - N*(1-Y[0,i]) <= Z[0,i])
        m.addConstr(0 <= Z[0,i])
    
    if model_type == LP:
        for i in range(improved_bound):
            m.addConstr(0 <= Y[0,i])
            m.addConstr(Y[0,i] <= 1)
    

In [ ]:
current_type = IP
start = 1000000
end = start + 300
data = []

for i in range(start, end+1):
    m = gp.Model()
    m.Params.LogToConsole = 0
    m.Params.Threads = 2
    
    model_one_factor_binary(i, current_type, m)
    
    for j in range(10):
        m.reset(1)
        m.optimize()

        if m.status == gp.GRB.OPTIMAL:
            factor = m.getVarByName("X").x
            correct = 0
            larger = 0
            if i % factor == 0:
                correct = 1
                if factor >= factor / i:
                    larger = 1


            data.append([i, factor, correct, larger, m.NumVars, m.NumIntVars, m.NumBinVars, m.NumConstrs, \
                         m.NodeCount, m.IterCount, m.runtime, m.IsMIP])
        elif m.status == gp.GRB.INFEASIBLE:
            data.append([i, "INFEASIBLE", "N/A", "N/A", m.NumVars, m.NumIntVars, m.NumBinVars, m.NumConstrs, \
                         m.NodeCount, m.IterCount, m.runtime, m.IsMIP])

In [ ]:
df = pd.DataFrame(data, columns=["N", "Factor", "Is Correct", "Is Larger Factor", \
                                 "Num Vars", "Num Integer Vars", "Num Binary Vars", "Num Constraints", \
                                 "Node Count", "Simplex Iterations Count", "Runtime", "Is MIP"])

In [ ]:
print(df)